In [1]:
## Homework #3 Data 445 Due Sept 24, 2021

In [2]:
## Q1 You are given a dataset having more variables than observations. Assuming that
## there seems to be a linear relationship between the target variable and the input variables in the
## dataset, why ordinary least squares (OLS) is a bad option to estimate the model parameters?
## Which technique would be best to use? Why?

## OLS is not a great option because there are so many variables tht you can get a high variance in the predicted
## values. This is due to small changes in the data set causing a large changes in the coefficients.
## Ridge regression is a good alternative for these situations, as it shrinks the coefficient values towards zero. Generating
## multiple sets of coefficient values (using different lambda values) allows a set of coefficients to be generated that
## minimizes variance in the predictions.

In [3]:
## Q2 For Ridge regression, if the regularization parameter, λ, is equal to 0, what are the implications?

## (f) (a) and (c)

## (a) Large coefficients in the linear model are not penalized & 
## (c)  The objective function is the same as ordinary least squares objective function.

In [4]:
## Q3 For Lasso Regression, if the regularization parameter, λ, is very high, which options are
## true? Select all that apply

## (f) (a) and (b)

## (a) Can be used to select important features of a dataset. &
## (b) Shrinks the coefficients of less important features to exactly 0

In [5]:
## Q4 Suppose you are using Ridge Regression and you notice that the training error and
## the validation error are almost equal and fairly high. Would you say that the model suffers from
## high bias or high variance? Should you increase the regularization parameter, λ, or reduce it?

In [6]:
## Q5a Load the data file to you S3 bucket. Using the pandas library, read the csv data
## file and create a data-frame called car price.

import pandas as pd
import boto3
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LassoCV, Lasso

## Defining the S3 bucket
s3 = boto3.resource('s3')
bucket_name = 'bonnieh-data-445-bucket'
bucket = s3.Bucket(bucket_name)

## Defining the file to be read from s3 bucket
file_key = 'CarPrice_Assignment.csv'

bucket_object = bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

##R Reading the csv file
car_price = pd.read_csv(file_content_stream)
car_price

,car_ID,symboling,CarName,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,wheelbase,...,enginesize,fuelsystem,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price
0,1,3,alfa-romero giulia,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495.0
1,2,3,alfa-romero stelvio,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500.0
2,3,1,alfa-romero Quadrifoglio,gas,std,two,hatchback,rwd,front,94.5,...,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500.0
3,4,2,audi 100 ls,gas,std,four,sedan,fwd,front,99.8,...,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950.0
4,5,2,audi 100ls,gas,std,four,sedan,4wd,front,99.4,...,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200,201,-1,volvo 145e (sw),gas,std,four,sedan,rwd,front,109.1,...,141,mpfi,3.78,3.15,9.5,114,5400,23,28,16845.0
201,202,-1,volvo 144ea,gas,turbo,four,sedan,rwd,front,109.1,...,141,mpfi,3.78,3.15,8.7,160,5300,19,25,19045.0
202,203,-1,volvo 244dl,gas,std,four,sedan,rwd,front,109.1,...,173,mpfi,3.58,2.87,8.8,134,5500,18,23,21485.0
203,204,-1,volvo 246,diesel,turbo,four,sedan,rwd,front,109.1,...,145,idi,3.01,3.40,23.0,106,4800,26,27,22470.0


In [7]:
## Q5b Using the wheelbase, enginesize, compressionratio, horsepower, peakrpm, citympg, and highwaympg as the 
## predictor variables, and price is the target variable. Do the following:

## Defining the variables
X = car_price[['wheelbase', 'enginesize', 'compressionratio', 'horsepower', 'peakrpm', 'citympg', 'highwaympg']]
Y = car_price['price']

## defining dataframe to store results for variable optimization
var_coef = []
##pd.DataFrame({'wb':np.repeat(np.nan, 1000), 'es':np.repeat(np.nan, 1000), 'cr':np.repeat(np.nan, 1000), 'hp':np.repeat(np.nan, 1000), 'prpm':np.repeat(np.nan, 1000), 'cmpg':np.repeat(np.nan, 1000), 'hmpg':np.repeat(np.nan, 1000)})
var_coef

for i in range (0, 1000):

    ## Split the data into train (80%) and test (20%)
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2)

    ## Using the train dataset:
    ## Estimate the optimal lambda using default values for lambda in scikit-learn and 5-folds. 
    ## Make sure to normalize the data (normalize = True).
    ## only do on train data, never on test
    lasso_cv = LassoCV(normalize = True, cv = 5).fit(X_train, Y_train)

    ## extracting the optimal alpha (lambda)
    cv_alpha = lasso_cv.alpha_
    cv_alpha
    
    ## Perform LASSO as a variable selector (using the optimal lambda from previous step 
    ## Make sure to normalize the data (normalize = True).
    lasso_md = Lasso(alpha = cv_alpha, normalize = True).fit(X_train, Y_train)
    lasso_md.coef_
    
    ## append coefficients to the dataframe
    var_coef.append(lasso_md.coef_)
    
    ##pd.concat([pd.DataFrame([lasso_md.coef_], columns = var_coef_df.columns), var_coef_df], ignore_index = True)
    
var_coef_df= pd.DataFrame(var_coef, columns = ['wb', 'es','cr','hp', 'prpm', 'cmpg', 'hmpg'])
var_coef_df

,wb,es,cr,hp,prpm,cmpg,hmpg
0,193.188967,114.774374,217.969415,47.520354,1.714653,-47.866599,0.000000
1,118.051697,107.394742,292.807424,52.059170,1.479092,-124.551240,-0.000000
2,180.211050,113.623802,282.948605,45.156092,2.402973,-86.463706,-0.000000
3,129.818599,116.660245,305.912093,55.460761,1.830296,-100.240525,-0.000000
4,179.523339,101.214286,284.140822,49.557371,1.742626,-138.982813,0.000000
...,...,...,...,...,...,...,...
995,171.797260,110.955228,233.229947,47.152957,2.148005,-83.423074,-0.000000
996,148.566276,112.083350,219.835460,46.827835,2.133882,-84.058473,-0.000000
997,139.425133,111.462374,301.461303,47.146137,1.994400,-125.808481,-0.000000
998,203.471064,108.483385,217.772013,47.385300,1.915059,-60.812334,-22.747867


In [14]:
## determining which variables have a coefficient of zero at least 500 of the 1000 runs

## counting the number of times there is a non-zero number for each column
var_coef_df.astype(bool).sum(axis=0)

## highwaympg is the only variable with less than 500 non-zero numbers (it is zero 745 times)

wb      1000
es      1000
cr      1000
hp      1000
prpm    1000
cmpg     987
hmpg     255
dtype: int64

In [13]:
## dropping the variables where there are less than 500 non-zero numbers
X_train = X_train.drop(columns = ['highwaympg'])
X_test = X_test.drop(columns = ['highwaympg'])
Y_train = Y_train.drop(columns = ['highwaympg'])
Y_test = Y_test.drop(columns = ['highwaympg'])
X_train

,wheelbase,enginesize,compressionratio,horsepower,peakrpm,citympg
117,108.0,134,7.0,142,5600,18
147,97.0,108,9.0,94,5200,25
183,97.3,109,9.0,85,5250,27
75,102.7,140,8.0,175,5000,19
36,96.5,92,9.2,76,6000,30
...,...,...,...,...,...,...
16,103.5,209,8.0,182,5400,16
159,95.7,110,22.5,56,4500,38
118,93.7,90,9.4,68,5500,37
109,114.2,120,8.4,97,5000,19


In [ ]:
## Q5d  Normalize the inputs variables of the train and test datasets using the L2 normalization. That is, for each input variable subtract the mean of that variable, then divide by
## the L2-norm of that variable.

In [ ]:
## Q5e Using the train dataset, build a linear regression model. After that, use this model
## to predict on the test dataset. Report the MSE of this model.

In [ ]:
## Q5f Using the train dataset, build a Ridge regression model as follows:
## (i) Using the train dataset, estimate the optimal lambda from the following set [0.001,
## 0.01, 0.1, 1, 10, 100] and using 5-folds.
## (ii) Repeat (i) 100, store the optimal lambda of each iteration.
## Using the most common lambda of the 100 optimal lambdas and the train dataset, build a
## Ridge regression model. After that, use this model to predict on the test dataset. Report
## the MSE of this model.